## Bitboard für Onitama



In [4]:
import numpy as np

#### Aufgabe 1:

Karte 8:
    
```
. . x
x o .
. . x

```
Eine einzelnen Position auf dem Board werden durch die Indizes des numpy-Arrays bezeichnet: 0 - 24

Schreibe eine Funktion n8, die jeder Zahl 0-24 eine Liste mit den bitboard-Positionen zuordnet, die durch Ausspielen der Karte 8 für weiß entstehen können. Weiß ist der Spieler unten, Schwarz ist der Spieler oben.


In [6]:
def n8(k):
    '''
    k: int, 0 - 24
    returns: Liste mit den Nachfolge-bitboards bei Spielen der Karte 8 an Position k
    '''  
    zeile = k // 5
    spalte = k % 5
    a = np.zeros((5,7), dtype = int)
    a[zeile,spalte] = 1
    a = bz(a)
    tmp = set()
    for x in [a << 1, a >> 8, a << 6]:
        x1b = board7(x)[:,:5]
        if bz(x1b) != 0:
            tmp.add(bz(x1b))
    return tmp
     
    

In [ ]:
def bz(a):
    ''' a : numpy-array
        returns: int, bitboard von a
    '''
    return int(''.join([str(x) for x in a.flatten()]),2)

def board7(k):
    ''' k: bitboard einer 5 x 7 boards
        returns: 5 x 7 numpy-Array
    '''
    bs = bin(k)[2:]
    bs = bs.rjust(35,'0')
    bs = bs[len(bs)-35:]
    i, j = 0, 0
    tmp = np.zeros((5, 7), dtype = int)
    for c in bs:
        tmp[i,j] = int(c)
        j = j+1
        if j > 6:
            j = 0
            i = i+1
    return tmp

def board5(k):
    ''' k: bitboard einer 5 x 5 boards
        returns: 5 x 5 numpy-Array
    '''
    bs = bin(k)[2:]
    bs = bs.rjust(25,'0')
    bs = bs[len(bs)-25:]
    i, j = 0, 0
    tmp = np.zeros((5, 5), dtype = int)
    for c in bs:
        tmp[i,j] = int(c)
        j = j+1
        if j > 4:
            j = 0
            i = i+1
    return tmp


a = np.zeros((5,7), dtype = int)
for k in range(25):
    for x in n8(k): 
        print(x,end= ' ')
    print()
 

### Aufgabe 2: 
Erweitere die Funktion von A1 auf alle Karten für weiß

In [12]:
def nw(k,id):
    '''
    k: int, 0 - 24, PositionsIndex
    id: int, 0 -15,  Kartenid
    returns: Liste mit den Nachfolge-bitboards bei Spielen der Karte id an Position k
    '''
    zeile = k // 5
    spalte = k % 5
    a = np.zeros((5,7), dtype = int)
    a[zeile,spalte] = 1
    a = bz(a)
    tmp = set()

    if id == 0: nbl = [a << 8, a >> 1, a >> 6]
    elif id == 1: nbl = [a << 7, a >> 6, a >> 8]
    elif id == 2: nbl = [a << 8, a << 2, a >> 8]
    elif id == 3: nbl = [a << 7, a << 1, a >> 7]
    elif id == 4: nbl = [a << 7, a << 1, a >> 1]
    elif id == 5: nbl = [a << 6, a << 1, a >> 1, a >> 6]
    elif id == 8: nbl = [a << 1, a >> 8, a << 6]  
    for x in nbl:
        x1b = board7(x)[:,:5]
        if bz(x1b) != 0:
            tmp.add(bz(x1b))
    return tmp

In [18]:
 
 
for x in nw(8,1): 
    print(board5(x))
    print()

[[0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]



### Board

Ein *board* ist ein 5x5 numpy-Array mit den Werten:
```
0 - frei
1 - Schüler Spieler 1
2 - Meister Spieler 1
3 - Schüler Spieler 2
4 - Meister Spieler2

```
 

 

In [63]:
def bz(a):
    ''' returned die bitzahl eines np-arrays'''
    return int(''.join([str(x) for x in a.flatten()]),2)

def board(k):
    ''' macht aus einer bitboard-Zahl eine np-Array'''
    bs = bin(k)[2:]
    bs = bs.rjust(25,'0')
    i, j = 0, 0
    tmp = np.zeros((5, 5), dtype = int)
    for c in bs:
        tmp[i,j] = int(c)
        j = j+1
        if j > 4:
            j = 0
            i = i+1
    return tmp

def showBoard(bz):
    ''' zeigt das Board einer bitzahl bz'''
    tmp = board(bz)
    for i in range(5):
        print(i,'|',*tmp[i])
    print('    =========')
    print('    A B C D E')
    print()
    
# mask = die gültigen Felder des Boards (ohne Trennfelder)
mask = bz(np.ones((5,5),dtype = int))

# w_start = die startfelder für weiß
w_start = np.zeros((5,5),dtype = int)
w_start[4]=1
w_start = bz(w_start)



### Aufgabe 1:
Gegeben die weißen Figuren in Grundstellung. Generiere alle Nachfolgestellungen für weiß bei Karte 8:

```
. . x
x o .
. . x

```

In [64]:
e = np.zeros((5,5), dtype = int)
e[4,4] = 1
e = bz(e)
e

1

In [102]:
e20, e21, e22, e23, e24 = 16, 8, 4, 2, 1
es = [e20,e21,e22,e23,e24]

wn = {}
wn[(e20,8)] = [e[k] for k in {20 << 4]
wn[(e21,8)] = [e21 << 1, e21 << 4]  
wn[(e22,8)] = [e22 << 1, e22 << 4] 
wn[(e23,8)] = [e23 << 1, e23 << 4] 
wn[(e24,8)] = [e23 << 1] 


# for e in es:
#     tmp = []
#     t = w_start & ~e   # die Figur wird gezogen
#     for x in wn[(e,8)]:
#         if not x & t:
#             tmp.append(t | x)
   
tmp = []

for e in es:
    t = w_start & ~e
    for x in wn[(e,8)]:
        if not (x & t):
            tmp.append(t | x)
for x in tmp:
    print(board(x))
    print()

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [0 1 1 1 1]]

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 1 1 1]]

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [1 1 0 1 1]]

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [1 1 1 0 1]]



In [112]:
e = {k : 2**(24-k) for k in range(25)}
e[0]

16777216

---

In [30]:
def makeBoard():
    return np.zeros((5, 7), dtype = int)
    
# def startBoard():
#     board = makeBoard()
#     board[4,0:5] = np.array([1,1,2,1,1])
#     board[0,0:5] = np.array([3,3,4,3,3])
#     return board


    
def startBoard():
    b = makeBoard()
    b[4,0:5] = 1

Das Startboard

In [27]:
b = makeBoard()
b[4,0:5] = 1
b = bz(b)
showBoard(b)

0 | 0 0 0 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 1 1 1 1 1
    A B C D E



In [9]:
board = startBoard()
showBoard(board)
 

0 | 3 3 4 3 3
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 1 1 2 1 1
    A B C D E



In [114]:
def boardzahlen(board):
    ''' returns int : zahl, die im bin-format das board repräsentiert'''
    bs1, bs2, bs3, bs4 = '', '', '', ''
    for j in range(4, -1, -1):
        for i in range(0, 5):
            bs1 += ['0', '1'][int(board[i, j] == 1)]
            bs2 += ['0', '1'][int(board[i, j] == 2)]
            bs3 += ['0', '1'][int(board[i, j] == 3)]
            bs4 += ['0', '1'][int(board[i, j] == 4)]
    return int(bs1, 2), int(bs2, 2), int(bs3, 2), int(bs4, 2)

In [116]:
board = startBoard()
bs1, bs2, bs3, bs4 = boardzahlen(board) 
bs1, bs2, bs3, bs4

(1081377, 1024, 17302032, 16384)

Wir betrachten den Bitstring, der durch die beiden Bitzahlen repräsentiert werden.

In [117]:
def bitboard(bz):
    bs = bitstr(bz)
    i, j = 4, 0
    board = np.zeros((5, 5), dtype = int)
    for c in bs[::-1]:
        board[i,j] = int(c)
        i-=1
        if i == -1:
            j += 1
            i = 4 
    showBoard(board)
    
def bitstr(bz):
    bitstr = bin(bz)[2:]
    return bitstr.rjust(25,'0')

In [63]:
print(bitstr(bs1), bitstr(bs2), bitstr(bs3), bitstr(bs4))
bitboard(bs1)
bitboard(bs2)
bitboard(bs3)
bitboard(bs4)

0000100001000000000100001 0000000000000010000000000 1000010000000001000010000 0000000000100000000000000
0 | 0 0 0 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 1 1 0 1 1
    A B C D E

0 | 0 0 0 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 1 0 0
    A B C D E

0 | 1 1 0 1 1
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 0 0 0
    A B C D E

0 | 0 0 1 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 0 0 0
    A B C D E



#### Modellierung des Spielstatus

Eine Spielstellung ist vollständig beschrieben durch 
a. die Lage auf dem Brett (die 4 Bitboards)
b. die Verteilung der Karten (id0 .. id4 in einer Liste)
c. wer dran ist (der untere Spieler 0 oder der obere 1)
d. der wievielte Zug es ist. Bei 200 endet das Spiel.


```
status = [boardzahlen, cards, playerid, turn]  
```

<img src = './img/onitama01.png' width = 500>

Diese Spielstellung wäre also repräsentiert durch



In [128]:
board = startBoard()
bz = boardzahlen(board)
cards = [2, 3, 14, 8, 10]
startstate = [bz, cards, player, turn]
startstate



[(1081377, 1024, 17302032, 16384), [2, 3, 14, 8, 10], 0, 0]

----

#### goaltest

Das Spiel ist zu Ende, wenn ein Meister geschlagen wurde, oder der Meister das Anfangsfeld des gegnerischen Meisters betritt. Nach 200 Zügen ist unentschieden.

In [101]:
M1 = 1024       # Startpos Meister 1
M2 = 16384      # Startpos Meister 2
bitboard(M2)

0 | 0 0 1 0 0
1 | 0 0 0 0 0
2 | 0 0 0 0 0
3 | 0 0 0 0 0
4 | 0 0 0 0 0
    A B C D E



In [126]:
s1, m1, s2, m2 = bz 

if M2 & m1:
    print('Meister 1 hat das Heimatfeld von Meister 2 erreicht')
if M1 & m2:
    print('Meister 2 hat das Heimatfeld von Meister 1 erreicht')
if not M1:
    print('Meister 1 wurde geschlagen')
if not M2:
    print('Meister 2 wurde geschlagen')
if turn >= 200:
    print('Unentschieden')


In [131]:
def goaltest(state):
    m1 = state[0][1]
    m2 = state[0][3]
    turn = state[3]
    return M2 & m1 or M1 & m2 or not M1 or not M2 or turn >= 200

In [132]:
goaltest(startstate)

False

## Neuer Ansatz